In [ ]:
import pandas as pd
import numpy as np
import ssl
import gensim
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ssl._create_default_https_context = ssl._create_unverified_context


class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss
        

class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        print(f'Epoch {self.epoch}')
        self.epoch += 1

        

# TF-IDF и Suggest

Этот семинар будет посвящен работе с текстами. Мы будем решать задачу о которой говорили на лекции: будем реализовывать suggest.

### План семинара


* [Задача suggest: многоклассовая классификация](#Задача-suggest)
* [Обучаем модель линейной регрессии](#Обучаем-модель-лог-регрессии)
    - Делаем бейзлайн
    - Улучшаем бейзлайн
    
* Обучаем word2vec
* Вычисляем признаки заголовков
* Смотрим метрику в нашей задаче

**Работа на семинаре**
* Реализуем tf-idf 

**Домашнее задание**
* Реализовать класс сервиса suggest-by-title

### Задача suggest

Подача без suggest:

<img src="https://ucarecdn.com/05ab935a-ff65-4d75-afcb-6eb0d71c5d44/" width="700">

Подача с suggest:
<img src="https://ucarecdn.com/5e1684f1-eec5-4054-9e43-08be8d9acbcb/" width="700">

Задача заключается в том, чтобы определить к какой категории относится объявление используя заголовок. Категорий в выборке 54, задача, соответственно многоклассовая классификация.

Задача многоклассовой классификации методом one-versus-all:

<img src="https://ucarecdn.com/2abe5812-3f6e-42cf-97a4-a35a6de58c6d/" width="534">

### Смотрим на данные

In [ ]:
data_train = pd.read_csv('suggest_train.csv', index_col=0)
data_test = pd.read_csv('suggest_test.csv', index_col=0)
w2v_train = pd.read_csv('unlabeled_data.csv', index_col=0)

In [ ]:
data_train.head()

### Обучаем модель линейной регрессии

#### Разбиваем данные на трейн и валидацию

Идеи:
- Частые слова зашумляют выборку;
- Редкие слова дают модели переобучиться;
- n-граммы позволят частично учитывать порядок;
- Регуляризация позволит уменьшить эффект переобучения;
- ???

In [ ]:
train_titles, val_titles, y_train, y_val = train_test_split(
    data_train['title'],
    data_train['category_id'],
    random_state=1
)

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer

In [ ]:
X_train = count_vectorizer.fit_transform(train_titles.values)
X_val = count_vectorizer.transform(val_titles.values)

In [ ]:
# print(len(count_vectorizer.vocabulary_))
# count_vectorizer.vocabulary_

In [ ]:
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs')
log_reg.fit(X_train, y_train)

In [ ]:
preds = log_reg.predict(X_val)
accuracy_score(preds, y_val)

### Учим w2v

Идеи:
- Обучить качественные векторные представление слов на неразмеченных данных;
- Получить из них вектора заголовков;
- Оценить адекватность;
- Оценить качество решения нашей задачи;
- Большая размерность позволит получить более качественные векторные представления;
- Сравнить CBOW и skip-gram;
- ???

In [ ]:
import re
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import SGDClassifier

WORD_PATTERN = '(?u)\\b\\w\\w+\\b'

reg_exp = re.compile(pattern=WORD_PATTERN)
w2v_train_data = pd.read_csv('unlabeled_data.csv',  index_col=0)
print(w2v_train_data.shape)
w2v_train_data.head()

In [ ]:
sentences = [reg_exp.findall(s.lower()) for s in w2v_train_data.title]
sentences[:3]

In [ ]:
w2v_model = Word2Vec(sg=1,)
w2v_model.build_vocab(sentences)
w2v_model.train(
    sentences,
    total_examples=w2v_model.corpus_count,
    epochs=20,
    compute_loss=True,
    callbacks=[LossLogger()]
)

In [ ]:
w2v_model.wv.similar_by_word('продам')

In [ ]:
w2v_model.wv.similar_by_word('айфон')

In [ ]:
class Word2VecTransformer:
    
    def __init__(self, w2v_model, word_pattern):
        
        self.w2v_model = w2v_model
        self.word_pattern = word_pattern
        self.re = re.compile(pattern=self.word_pattern)
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        
        X_transformed = np.zeros((len(X), self.w2v_model.wv.vector_size))
        for i, title in enumerate(X):
            
            title_vector = np.zeros((self.w2v_model.wv.vector_size,))
            tokens = self.re.findall(title.lower())
            for token in tokens:
                if token in self.w2v_model.wv.key_to_index:
                    title_vector += self.w2v_model.wv.get_vector(token)
                    
            X_transformed[i] = title_vector
        
        return X_transformed

In [ ]:
w2v_transformer = Word2VecTransformer(w2v_model=w2v_model, word_pattern=WORD_PATTERN)

train_w2v = w2v_transformer.transform(train_titles.values)
val_w2v = w2v_transformer.transform(val_titles.values)

In [ ]:
log_reg = SGDClassifier()
log_reg.fit(train_w2v, y_train)

In [ ]:
preds = log_reg.predict(val_w2v)
accuracy_score(preds, y_val)

### Работа на семинаре: [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class TFIDFVectorizer(BaseEstimator, TransformerMixin):
    
    def __init__():
        pass
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        pass

### Работа на семинаре: suggest на подаче

* Реализовать [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) transformer
* Обучить линейную модель на признаках TF-IDf сравнить с реализацией из sklearn
    - Разобраться с каждым гиперпараметром реализации TF-IDf из sklearn
    
* Обучить word2vec на данных data/unlabeled_data.csv
* Попробовать улучшить результат который получили с помощью w2v

### Домашнее задание

Теоритическая часть (обязательно).

1. Посмотреть видео на [3BlueBrowb](https://youtu.be/aircAruvnKk)

Практическая часть.

**Реализовать класс Suggester, который возвращает от 1 до 5 наиболее вероятных категорий по введённой строке.**

accuracy по top1 > 0.78

In [ ]:
from typing import List

class Suggester:
    
    def __init__(self, max_suggest_count=5, default_suggest=''):
        self.max_suggest_count = max_suggest_count
        self.default_suggest = default_suggest
        
    def suggest(self, title: str):
        print('Для дома и дачи|Посуда и товары для кухни|Посуда')
        print('Хобби и отдых|Коллекционирование|Другое')
    
    def test_suggest(self):
        
        title = input()
        while title != 'stop':
            
            suggest = self.suggest(title)

            if suggest:
                print(suggest)
            else:
                print(self.default_suggest)
            
            title = input()

In [ ]:
sug = Suggester()
sug.test_suggest()